# Coordinate Reference System (CRS) Management

Rasterix uses [XProj](https://xproj.readthedocs.io/en/latest/) to store and manage the CRS of a dataset

In [ ]:
# GDAL settings for remote data
import os

import xarray as xr
import xproj

import rasterix

os.environ["VSICURL_PC_URL_SIGNING"] = "YES"
os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"

xr.set_options(display_expand_indexes=True);

In [ ]:
url = "https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif"
da = xr.open_dataarray(url, engine="rasterio")

da = rasterix.assign_index(da).set_xindex("spatial_ref", xproj.CRSIndex)

da

In [ ]:
# Make sure we can save this back to a geotiff
# Hmmm, this warning isn't clear to me
da.rio.to_raster("/tmp/S_20240101_concentration_v3.0.tif")

In [ ]:
!wget -nc https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif
!ls -l S_20240101_concentration_v3.0.tif

In [ ]:
# Are the tiffs the same?
!ls -l /tmp/S_20240101_concentration_v3.0.tif

## Two rasters with same GeoTransform but different CRS

Here is a case for needing to account for both affine and CRS. [Sentinel-2](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) is delivered on the MGRS grid, which divides UTM Zones into smaller units. Adjacent zones have the same affines, and therefore appear to have the same coordinates. But the CRS differentiates them.

We want Xarray to raise an error in this case before attempting any computations.

In [ ]:
def load_raster(href, overview_level=3, masked=True):
    da = xr.open_dataarray(
        href, engine="rasterio", masked=masked, open_kwargs=dict(overview_level=overview_level), chunks="auto"
    ).squeeze()

    return rasterix.assign_index(da).set_xindex("spatial_ref", xproj.CRSIndex)


url = "https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/DS/2025/04/14/S2C_MSIL2A_20250414T190931_N0511_R056_T10TDS_20250415T001314.SAFE/GRANULE/L2A_T10TDS_A003172_20250414T191846/IMG_DATA/R10m/T10TDS_20250414T190931_B04_10m.tif"
da1 = load_raster(url)

url = "https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/11/T/MM/2025/04/13/S2B_MSIL2A_20250413T184919_N0511_R113_T11TMM_20250413T224733.SAFE/GRANULE/L2A_T11TMM_A042324_20250413T185100/IMG_DATA/R10m/T11TMM_20250413T184919_B04_10m.tif"
da2 = load_raster(url)

da1 + da2